# Purpose

Find a way to get data from Plugshare.com since they're not responding to my API access request. The comments and metadata from stations across different networks should be extremely useful in diagnosing electrical and non-electrical customer experience issues.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from rich import print
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests

from evlens.data.plugshare import Scraper

from dotenv import load_dotenv
load_dotenv(override=True)

from evlens.logs import setup_logger
logger = setup_logger("Notebook-0.1")
logger.info("TEST!")

2024-06-13_T20_30_55EDT: INFO (Notebook-0.1:L18) - TEST!


# Testing our custom scraper

In [2]:
# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784
url = f"https://www.plugshare.com/location/{TEST_LOCATION}"

chrome_options = Options()
# chrome_options.add_argument('--headless=new')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options=chrome_options)

driver.get(url)

wait = WebDriverWait(driver, 5)

# Wait for the cookie dialog to appear
try:
    iframe = wait.until(EC.visibility_of_element_located((
        By.ID,
        "global-consent-notice"
    )))
    logger.info("Found the banner!")
    
except (NoSuchElementException, TimeoutException) as e1:
    logger.warning("No cookie dialog iframe found, moving on!")

2024-06-16_T14_00_38EDT: INFO (Notebook-0.1:L28) - Found the banner!


In [68]:
results = driver.find_elements(By.ID, "global-consent-notice")
len(results)

1

In [74]:
main_iframe = results[0]
main_id = main_iframe.id
main_id

'f.16AFDF4310127F02E90EF1E40AFE08C4.d.E4B2C29C6FE2C310BB0D15345750526D.e.162'

In [75]:
for i, IFRAME in enumerate(IFRAMES):
    for j, iframe in enumerate(IFRAME):
        if iframe.id == main_id:
            print(f"Found `main_id` in IFRAMES[{i}][{j}]")

Found `main_id` in IFRAMES[0][8]

In [79]:
# Does the dialog iframe have other iframes?

driver.switch_to.frame(main_iframe)
driver.find_elements(By.TAG_NAME, "iframe")

# No child iframes found...

[]

In [81]:
# Can I find the Accept button?
main_buttons = driver.find_elements(By.TAG_NAME, "button")
len(main_buttons)

1

In [83]:
# There's only one button
main_button = main_buttons[0]
main_button.id

'f.3390C00A4C87815493BD64831889EBE0.d.2E17957C23E49B6AF2E621E995A036DA.e.12'

In [84]:
# What if we just click it and accept cookies?
main_button.click()

In [65]:
# Figure out clicking that damned iframe...
IFRAMES = []
driver.switch_to.default_content()
def find_all_iframes(driver, iframes):
    iframes_found = driver.find_elements(By.TAG_NAME, "iframe")
    iframes.append(iframes_found)
    for iframe in iframes_found:
        # Your sweet business logic applied to iframe goes here.
        driver.switch_to.frame(iframe)
        find_all_iframes(driver, iframes)
        driver.switch_to.parent_frame()
        
find_all_iframes(driver, IFRAMES)

driver.switch_to.default_content()
IFRAMES = [i for i in IFRAMES if len(i) > 0]
len(IFRAMES)

14

In [66]:
[len(i) for i in IFRAMES]

[18, 1, 11, 2, 6, 2, 2, 19, 9, 3, 1, 16, 1, 1]

Enumerating based off of the list at index `i` in `IFRAMES[i]`:

1. 
2. 

In [28]:
# Switch back to main frame
driver.switch_to.default_content()

# Exit login dialog
try:
    # Wait for the exit button
    wait = WebDriverWait(driver, 1)
    esc_button = wait.until(EC.visibility_of_element_located((
        By.XPATH,
        # "//*[@id=\"dialogContent_authenticate\"]/button/md-icon" # old
        "//*[@id=\"dialogContent_authenticate\"]/button" # from chrome
    )))
    esc_button.click()
    logger.info("Found the login escape button and clicked it!")

except (NoSuchElementException, TimeoutException):
    logger.error("Login dialog exit button not found.")

except Exception as e:
    logger.error(f"Unknown error trying to exit login dialog: {e}")

2024-06-14_T09_23_20EDT: ERROR (Notebook-0.1:L14) - Login dialog exit button not found.


In [29]:
# Do a single element scrape
try: ## FIND STATION NAME
    wait.until(EC.visibility_of_element_located((
        By.XPATH,
        "//*[@id=\"display-name\"]/div/h1"
    )))
    name = driver.find_element(
        By.XPATH,
        "//*[@id=\"display-name\"]/div/h1"
        ).text
except:
    logger.error("Station name error", exc_info=True)
    name = np.nan
    
name

2024-06-14_T09_24_24EDT: ERROR (Notebook-0.1:L12) - Station name error
Traceback (most recent call last):
  File "/var/folders/98/fhwnl49n19l_xywxzghbm4jm0000gn/T/ipykernel_58677/2682536154.py", line 3, in <module>
    wait.until(EC.visibility_of_element_located((
  File "/Users/davemcrench/Documents/Projects/evlens/.venv/lib/python3.11/site-packages/selenium/webdriver/support/wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



nan

In [25]:
driver.quit()

In [28]:
# Why is it taking so long to even *start* trying to exit the login dialog?!
s = Scraper(timeout=3)

# Scrape only one location that I can test via browser
df = s.scrape_plugshare_locations(TEST_LOCATION, TEST_LOCATION)
df.info()
df.head()

In [19]:
df.loc[0, 'Comments']

'J-1772 1 Plug 7 kW\n1 Station\nElectrify America, CCS/SAE 3 Plugs 350 kW\n3 Stations\nElectrify America, , Jun 7, 2024\ncheck_circleJ C\nBMW iX 2024, Jun 5, 2024\ncheck_circleKJva\nMercedes EQE 350350 Kilowatts, May 30, 2024\ncheck_circleKJva\nMercedes EQE 350350 Kilowatts, May 24, 2024\ncheck_circleBEV\nChevrolet Blazer EV 2024\nSo far so good! Screen is cracked on the charger, difficult to read the prompts… but I guess it doesn’t matter if it charges!, May 24, 2024\ncheck_circleKMac\nKia EV6 2022235 Kilowatts\nScreen on 1 not working but can charge through the app, 0.08 mi\n9.3 Springfield Town Center - Target - East Lot (2)\nJ-1772, 0.18 mi\n4.8 Springfield Town Center - LA Fitness\nJ-1772, 0.22 mi\n1 Springfield Town Center - Frontier Garage\nJ-1772, 0.22 mi\n10 Springfield Town Center - Target - West Lot\nNACS (Tesla), 0.23 mi\n4.7 Springfield Town Center - Maggianos\nJ-1772, , , '